In [1]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from transformers import pipeline


import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Preprocess messages
nltk.download('punkt')
nltk.download('stopwords')

# Load dataset
data = pd.read_csv('ScamDataset', sep='\t', names=['Label', 'message'])

stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text)
    stemmed = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(stemmed)

data['processed_message'] = data['message'].apply(preprocess)
texts = data['processed_message'].tolist()

data['Label'] = data['Label'].map({'normal': 0, 'fraud': 1})  # Adjust as necessary based on your actual labels
labels= data['Label'].tolist()


conversations = texts


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /Users/nicholas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicholas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Split the data into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1)  # 10% for validation

def tokenize_data(tokenizer, data):
    return tokenizer(texts, padding=True, truncation=True, max_length=256, return_tensors="pt")


In [3]:

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Set the padding token to an existing special token, such as 'eos_token'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)# Adjust num_labels based on your classification needs


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:43<00:00, 10.89s/it]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

train_encodings = tokenize_data(tokenizer, train_data)
val_encodings = tokenize_data(tokenizer, val_data)


In [5]:

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {key: torch.tensor(val).to(device) for key, val in encodings.items()}
        self.labels = torch.tensor(labels).to(device)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)
train_dataset = CustomDataset(train_encodings, train_data['Label'].tolist())
val_dataset = CustomDataset(val_encodings, val_data['Label'].tolist())



/var/folders/6p/1lxkv5mn6mld1pk9xvwqqr_00000gn/T/ipykernel_73214/2852898923.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encodings = {key: torch.tensor(val).to(device) for key, val in encodings.items()}


In [6]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    gradient_accumulation_steps=2,  # This effectively doubles the effective batch size
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


RuntimeError: MPS backend out of memory (MPS allocated: 17.93 GB, other allocations: 384.00 KB, max allowed: 18.13 GB). Tried to allocate 224.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

